In [1]:
# Need this for matrix multiplication to work for some reason lol
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [31]:
import torch
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from vae import VAE
import numpy as np
from scipy.linalg import svd, null_space

In [3]:
vae = torch.load("vae.model")
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [4]:
# Get all the weight/bias matrices.
w1 = vae.fc1.weight.cpu().detach().numpy()
b1 = vae.fc1.bias.cpu().detach().numpy()
w2 = vae.fc2.weight.cpu().detach().numpy()
b2 = vae.fc2.bias.cpu().detach().numpy()
w3 = vae.fc31.weight.cpu().detach().numpy()
b3 = vae.fc31.bias.cpu().detach().numpy()

print("Weights shape", w1.shape, w2.shape, w3.shape)
print("Biases shape", b1.shape, b2.shape, b3.shape)

Weights shape (512, 784) (256, 512) (2, 256)
Biases shape (512,) (256,) (2,)


In [49]:
def invert_leaky_relu_(scalar, slope=0.1):
    """
    Inverts a leaky relu transformation
    """
    return scalar if scalar >= 0 else scalar / slope


def sample_preimage(W, b, z, n, noise_level=0.001, as_list=False):
    """
    Finds n points x_1, ..., x_n such that W x_i + b = z for all i
    """
    # Subtract b from eq
    new_b = z-b
    # Compute pseudoinverse
    pinv = np.linalg.pinv(W)
    # Get the "central" x which solves the equation (i.e., no null space vectors)
    x_orig = pinv @ new_b
    # Compute the null_space of W (transpose to get right orientation)
    ns = null_space(W).T
    # Compute some Gaussian noise, and then scale by noise_level
    noise = np.random.randn(n, ns.shape[0]) * noise_level
    # Noise_null_space matrix
    noise_null_mat = noise @ ns
    # Setup the array where our samples will live in
    new_points = np.empty((n, x_orig.shape[0]))
    # Generate our samples
    for i in range(n):
        new_points[i] = x_orig + noise_null_mat[i]
        
    # Check that our sampled points are actually correct
    #assert np.allclose((new_points @ W.T) + b, np.repeat(np.expand_dims(z, 0), n, axis=0))
    
    # Return the original x, as well as the sampled points
    if as_list:
        return x_orig, list(new_points)
    else:
        return x_orig, new_points

# Create the vectorized version of inverse_leaky_relu
invert_leaky_relu = np.vectorize(invert_leaky_relu_)

def generate_preimage_points(z1, w1, w2, w3, b1, b2, b3, n_splits=3):
#     assert(len(w_tup) == len(b_tup))
    
#     s_final = np.empty((n_splits ** len(w_tup), ))
#     for i in range(len(w_tup)):
#         for n in range(n_splits ** i):
#             _, s = sample_preimage(w_tup[-(i+1)], b_tup[-(i+1)], z1, n_splits)
#             s1 = invert_leaky_relu(s)
#             print()
    
    # Generate first set of preimage points
    _, s1 = sample_preimage(w3, b3, z1, n_splits)
    # Invert leaky relu and convert to list
    s1 = list(invert_leaky_relu(s1))

    # Generate second set of preimage points
    s2 = np.empty((n_splits * n_splits, w2.shape[1]))
    for i, z1 in enumerate(s1):
        _, s2_single = sample_preimage(w2, b2, z1, n_splits)
        # Fill in our samples
        s2[n_splits*i:n_splits*(i+1)] = s2_single
    # Invert leaky relu and convert to list
    s2 = list(invert_leaky_relu(s2))
    
    # Generate 3rd set of preimage points
    s3 = np.empty((n_splits ** 3, w1.shape[1]))
    for i, z2 in enumerate(s2):
        _, s3_single = sample_preimage(w1, b1, z2, n_splits)
        s3[n_splits*i:n_splits*(i+1)] = s3_single
    
    # We have our preimage points, so we can return.
    return s3.reshape((s3.shape[0], 1, 28, 28))
    
def generate_preimage_points_2(z1, w1, w2, b1, b2, n_splits=3, h1=None):
#     assert(len(w_tup) == len(b_tup))
    
#     s_final = np.empty((n_splits ** len(w_tup), ))
#     for i in range(len(w_tup)):
#         for n in range(n_splits ** i):
#             _, s = sample_preimage(w_tup[-(i+1)], b_tup[-(i+1)], z1, n_splits)
#             s1 = invert_leaky_relu(s)
#             print()
    # For the second one, invert leaky_relu
    
    z1 = invert_leaky_relu(z1)
    # Generate first set of preimage points
    orig, s1 = sample_preimage(w2, b2, z1, n_splits)
    if h1 is not None:
        print(np.linalg.norm(orig - h1))
    # Invert leaky relu and convert to list
    s1 = list(invert_leaky_relu(s1))

    # Generate second set of preimage points
    s2 = np.empty((n_splits ** 2, w1.shape[1]))
    for i, z1 in enumerate(s1):
        _, s2_single = sample_preimage(w1, b1, z1, n_splits)
        # Fill in our samples
        s2[n_splits*i:n_splits*(i+1)] = s2_single
    # Invert leaky relu and convert to list
#     s2 = list(invert_leaky_relu(s2))
    
    # We have our preimage points, so we can return.
    return s2.reshape((s2.shape[0], 1, 28, 28))

def generate_preimage_points_3(z1, w1, b1, n_splits=3):
#     assert(len(w_tup) == len(b_tup))
    
#     s_final = np.empty((n_splits ** len(w_tup), ))
#     for i in range(len(w_tup)):
#         for n in range(n_splits ** i):
#             _, s = sample_preimage(w_tup[-(i+1)], b_tup[-(i+1)], z1, n_splits)
#             s1 = invert_leaky_relu(s)
#             print()
    # For the second one, invert leaky_relu
    
    z1 = invert_leaky_relu(z1)
    # Generate first set of preimage points
    _, s1 = sample_preimage(w1, b1, z1, n_splits)
    
    # We have our preimage points, so we can return.
    return s1.reshape((s1.shape[0], 1, 28, 28))

In [ ]:
def compute_dist(v1, v2):
    

In [52]:
encoded_data = torch.load('encoded_data.pt')
idx = 51
z = encoded_data['mu'][idx].detach().numpy()
h1 = encoded_data['h1'][idx].detach().numpy()
h2 = encoded_data['h2'][idx].detach().numpy()
print(z.shape, w1.shape, w2.shape, w3.shape, b1.shape, b2.shape, b3.shape)
x_inv1 = generate_preimage_points(z, w1, w2, w3, b1, b2, b3)
x_inv2 = generate_preimage_points_2(h2, w1, w2, b1, b2, n_splits=5, h1=h1)
x_inv3 = generate_preimage_points_3(h1, w1, b1, n_splits=25)

(2,) (512, 784) (256, 512) (2, 256) (512,) (256,) (2,)
6.879941900608678


In [8]:
save_image(torch.tensor(x_inv1), './inverted_vae_output_t1.png')
save_image(torch.tensor(x_inv2), './inverted_vae_output_t2.png')
save_image(torch.tensor(x_inv3), './inverted_vae_output_t3.png')

(27, 1, 28, 28)